# Effect of $\epsilon$ on Average Treatment Effect on IHDP

In [1]:
import copy

import numpy as np
import matplotlib.pyplot as plt
import torch

from misc.agm import calibrateAnalyticGaussianMechanism

%matplotlib inline

# set random seed
np.random.seed(1)
torch.manual_seed(1)

In [2]:
# no. samples for fitting, no. samples for estimating, no. of draws of z
nf = 500
nt = 500
nd = 1

# privacy parameters
epses = [0, 0.2, 0.4, 0.6, 0.8, 0.99]
delta = torch.tensor(1e-6)

# regularisation coefficient
reg_co = 0.1

## Preprocess IHDP data

In [3]:
# load training data
train = {}

with np.load('data/IHDP-1000/ihdp_npci_1-1000.train.npz') as data:
    for i in data.files:
        # convert all arrays to torch tensors
        train[i] = torch.tensor(data[i], dtype=torch.float64)
        
# ate is the true average treatment effect
# yf is the noisy factual outcome
# need only, ate, yf, t and x

# swap axis and preprocess x
for i in  ['yf', 't', 'x']:
    if i != 'x':
        train[i] = train[i].transpose(0, 1)
    else:
        train[i] = train[i].permute(2, 0, 1)
        
# get no. experiments and dim
ne, _, d = train['x'].shape

# change ne if required
# ne = 250

# generate X, T, Y through subsampling
Y_train, X_train = [], []
T_train = torch.stack(
    [torch.cat([torch.ones(int(nf/2), dtype=torch.float64), torch.zeros(int(nf/2), dtype=torch.float64)])] * ne
)

for i in range(ne):
    # get indices for t=1 and t=0
    t1_idx = train['t'][i, :].nonzero().squeeze()
    t0_idx = (1 - train['t'][i, :]).nonzero().squeeze()
                 
    # subsample n indices, n/2 for t=1 and n/2 for t=0
    sam_idx = np.hstack([np.random.choice(t1_idx, int(nf/2)), np.random.choice(t0_idx, int(nf/2))])
    
    Y_train.append(train['yf'][i, sam_idx])
    X_train.append(train['x'][i, sam_idx, :])
    
# convert to torch tensors 
Y_train, X_train = torch.stack(Y_train), torch.stack(X_train)

# permute data
# permute indices
perm = torch.stack([torch.randperm(nf) for i in range(ne)])

# create auxiliary indices
idx = torch.arange(ne)[:, None]

# permute X_train, T_train, Y_train
X_train = X_train[idx, perm]
T_train = T_train[idx, perm]
Y_train = Y_train[idx, perm]

In [4]:
# load test data
test = {}

with np.load('data/IHDP-1000/ihdp_npci_1-1000.test.npz') as data:
    for i in data.files:
        # convert all arrays to torch tensors
        test[i] = torch.tensor(data[i], dtype=torch.float64)
        
# ate is the true average treatment effect
# yf is the noisy factual outcome
# need only, ate, yf, t and x

# swap axis and preprocess x
for i in  ['yf', 't', 'x']:
    if i != 'x':
        test[i] = test[i].transpose(0, 1)
    else:
        test[i] = test[i].permute(2, 0, 1)
        
# get no. experiments and dim
ne, _, d = test['x'].shape

# change ne if required
# ne = 250

# generate X_test, T_test, Y_test through subsampling
Y_test, X_test = [], []
T_test = torch.stack([torch.cat(
    [torch.ones(int(nt/2), dtype=torch.float64), torch.zeros(int(nt/2), dtype=torch.float64)])] * ne
)

for i in range(ne):
    # get indices for t=1 and t=0
    t1_idx = test['t'][i, :].nonzero().squeeze()
    t0_idx = (1 - test['t'][i, :]).nonzero().squeeze()
                 
    # subsample n indices, n/2 for t=1 and n/2 for t=0
    sam_idx = np.hstack([np.random.choice(t1_idx, int(nt/2)), np.random.choice(t0_idx, int(nt/2))])
    
    Y_test.append(test['yf'][i, sam_idx])
    X_test.append(test['x'][i, sam_idx, :])
    
# convert to torch tensors 
Y_test, X_test = torch.stack(Y_test), torch.stack(X_test)

# permute data
# permute indices
perm = torch.stack([torch.randperm(nt) for i in range(ne)])

# create auxiliary indices
idx = torch.arange(ne)[:, None]

# permute X_train, T_train, Y_train
X_test = X_test[idx, perm]
T_test = T_test[idx, perm]
Y_test = Y_test[idx, perm]

In [5]:
# concatenate train and test
X_all = torch.cat([X_train, X_test], 1)
T_all = torch.cat([T_train, T_test], 1)
Y_all = torch.cat([Y_train, Y_test], 1)

## Define model and method

In [6]:
class Log_Reg(torch.nn.Module):
    '''
    Logistic Regression
    '''
    def __init__(self, D_in, D_out):
        super(Log_Reg, self).__init__()
        self.linear = torch.nn.Linear(D_in, D_out, bias=False)
        
    def forward(self, x):
        y_pred = torch.sigmoid(self.linear(x))
        return y_pred

In [7]:
def IPW_PPS_Out(X, T, Y, epses, delta, reg_co, nf):
    '''
    average treatment effect with inverse propensity weighting using private propensity scores
    '''
    # get # experiments, # samples, # dimensions
    ne, ns, dim = X.shape

    ################
    # process data #
    ################

    # get Y0 and Y1
    Y0 = Y * (1 - T)
    Y1 = Y * T
    
    # split data
    # get splits
    fit_split = nf
    est_split = ns - nf

    # create auxiliary indices
    idx = torch.arange(ne)[:, None]

    # split X into fit, estimate splits
    X_s0 = X[:, :fit_split]
    X_s1 = X[:, fit_split:]

    # expand dim of T to allow multiplication with X
    T_ex_dim = T.reshape(ne, ns, 1)

    # split X0 and X1 into fit, estimate splits
    X0_s1 = (X * (1 - T_ex_dim))[:, fit_split:]
    X1_s1 = (X * T_ex_dim)[:, fit_split:]

    # split T into fit, estimate splits
    T_s0 = T[:, :fit_split]
    T_s1 = T[:, fit_split:]
        
    # split Y0 and Y1 into fit, estimate splits
    Y0_s0 = Y0[:, :fit_split]
    Y1_s0 = Y1[:, :fit_split]

    Y0_s1 = Y0[:, fit_split:]
    Y1_s1 = Y1[:, fit_split:]
    
    ##############
    # fit models #
    ##############
    
    models = []
    
    for expm in range(ne):
        X = X_s0[expm]
        T = T_s0[expm][:, None]
        model = Log_Reg(dim, 1).double()
        opt = torch.optim.LBFGS(model.parameters(), max_iter=100)

        # define first-order oracle for lbfgs
        def closure():
            if torch.is_grad_enabled():
                opt.zero_grad()
            outputs = model(X)
            for weights in model.parameters():
                loss = torch.nn.functional.binary_cross_entropy_with_logits(outputs, T) + 0.5 * reg_co * weights.norm(2).pow(2)
            if loss.requires_grad:
                loss.backward()
            return loss

        opt.step(closure)

        models.append(model)

    #############################
    # estimate treatment effect #
    #############################

    # initialise pi_hat dictionaries
    pi_hats = {}
    
    # initialise e dictionary
    e = {}
    
    # intialise sigma dictionary
    sig_d = {}

    # get estimated propensity scores
    pi_hats[0] = torch.stack(
        [models[i](X_s1[i]).squeeze() for i in range(ne)]
    )

    # perturb model and get relevant quantities
    for eps in epses[1:]:
        # define sensitivity for log reg
        s_w = 2.0 / (fit_split * reg_co)

        # define sigma for log reg
        sigma = np.sqrt(
            2 * np.log(1.25 / delta) + 1e-10
        ) * (s_w / (eps / 2))
        sigma_2 = sigma ** 2

#         # analytic gaussian mechanism
#         sigma = calibrateAnalyticGaussianMechanism(eps, delta, s_w)
#         sigma_2 = sigma ** 2

        # define z distribution for log reg
        z_dist = torch.distributions.normal.Normal(
            torch.tensor(0.0, dtype=torch.float64),
            torch.tensor(sigma, dtype=torch.float64),
        )

        # draw z for log reg
        z_vecs = z_dist.sample((ne, dim))

        # create temp models
        models_ = copy.deepcopy(models)

        # initialise list for privatised estimated propensity scores
        pi_hats[eps] = []

        # perturb weights with z_vecs
        for i in range(ne):
            model_temp = models_[i]
            model_temp.linear.weight.data.add_(
                z_vecs[i]
            )
            pi_hats[eps].append(
                model_temp(X_s1[i]).squeeze()
            )

        # reshape stacked privatised estimated propensity scores
        pi_hats[eps] = torch.stack(pi_hats[eps])
                        
        # max of abs of Y1_s1 / propensity score for each experiment
        max_abs_Y1_s1_div_ps = torch.max(
            torch.abs(Y1_s1) / ((ns - nf) * pi_hats[eps]), 1
        )[0]
        
        # max of abs of Y0_s1 / (1 - propensity score) for each experiment
        max_abs_Y0_s1_div_1_m_ps = torch.max(
            torch.abs(Y0_s1) / ((ns - nf) * (1 - pi_hats[eps])), 1
        )[0]
        
        # hstack max_abs_Y_s1_div_ps and max_abs_Y_s1_div_1_m_ps
        max_abs_all = torch.stack(
            (max_abs_Y1_s1_div_ps, max_abs_Y0_s1_div_1_m_ps), 1
        )
        
        # replace inf/nan with 1e20 for stability
        max_abs_all[torch.isfinite(max_abs_all) == 0] = 1e20
            
        # define sensitivity for estimation
        s_e = 2 * torch.max(max_abs_all, 1)[0]
        
        # define sigma for estimation
        sigma_e = np.sqrt(
            2 * np.log(1.25 / delta) + 1e-10
        ) * (s_e / (eps / 2))
        sig_d[eps] = sigma_e.detach().numpy()
        sigma_e_2 = sigma_e ** 2
        
#         # analytic gaussian mechanism
#         sigma_e = calibrateAnalyticGaussianMechanism(eps, delta, s_e)
#         sigma_e_2 = sigma_e ** 2

        # define e distribution for estimation
        e_dist = torch.distributions.multivariate_normal.MultivariateNormal(
            torch.tensor([0.0], dtype=torch.float64),
            torch.diag(sigma_e)
        )

        # draw e for estimation
        e[eps] = e_dist.sample().reshape(ne)
    
    # get treatment effects
    # true
    te = {}
    # empirical means and std of means of ERM + private ERM
    te_hats = {'means': [], 'stds': []}
    # means and std of means of privatised te_hats
    te_hats_p = {'means': [], 'stds': []}
                
    for key in pi_hats.keys():
        # empirical estimate for noiseless case
        # reduce_mean from (ne, est_split) tensor to (ne , 1) matrix
        te_hats_ = torch.mean(
            Y1_s1 / pi_hats[key] - Y0_s1 / (1 - pi_hats[key]),
            1,
        )
        te_hats['means'].append(
            te_hats_.detach().numpy()
        )
        te_hats['stds'].append(
            te_hats_.std().detach().numpy()
        )
        try:
            te_hats_p_ = te_hats_ + e[key]
            te_hats_p['means'].append(
                te_hats_p_.detach().numpy()
            )
            te_hats_p['stds'].append(
                te_hats_p_.std().detach().numpy()
            )
        except KeyError:
            # fill first row for later
            te_hats_p['means'].append(
                te_hats_.detach().numpy()
            )
            te_hats_p['stds'].append(
                te_hats_.std().detach().numpy()
            )
        
    te_hats['means'] = np.array(te_hats['means'])
    te_hats['stds'] = np.array(te_hats['stds'])
    te_hats_p['means'] = np.array(te_hats_p['means'])
    te_hats_p['stds'] = np.array(te_hats_p['stds'])

    return te, te_hats, te_hats_p, sig_d

In [8]:
def IPW_PPS_Obj(X, T, Y, epses, delta, reg_co, nf):
    '''
    average treatment effect with inverse propensity weighting using private propensity scores
    '''
    # get # experiments, # samples, # dimensions
    ne, ns, dim = X.shape

    # objective perturbation constants
    L = 1 # see from derivation, also http://proceedings.mlr.press/v32/jain14.pdf
    R2 = 1 # as norm is bounded by 1
    c = 0.25
    
    ################
    # process data #
    ################

    # get Y0 and Y1
    Y0 = Y * (1 - T)
    Y1 = Y * T
    
    # split data
    # get splits
    fit_split = nf
    est_split = ns - nf

    # create auxiliary indices
    idx = torch.arange(ne)[:, None]

    # split X into fit, estimate splits
    X_s0 = X[:, :fit_split]
    X_s1 = X[:, fit_split:]

    # expand dim of T to allow multiplication with X
    T_ex_dim = T.reshape(ne, ns, 1)

    # split X0 and X1 into fit, estimate splits
    X0_s1 = (X * (1 - T_ex_dim))[:, fit_split:]
    X1_s1 = (X * T_ex_dim)[:, fit_split:]

    # split T into fit, estimate splits
    T_s0 = T[:, :fit_split]
    T_s1 = T[:, fit_split:]
        
    # split Y0 and Y1 into fit, estimate splits
    Y0_s0 = Y0[:, :fit_split]
    Y1_s0 = Y1[:, :fit_split]

    Y0_s1 = Y0[:, fit_split:]
    Y1_s1 = Y1[:, fit_split:]
    
    ##############
    # fit models #
    ##############

    z_dist = torch.distributions.normal.Normal(
                torch.tensor(0.0, dtype=torch.double),
                torch.tensor(1.0, dtype=torch.double),
                )
    
    models = {}
    
    for eps in epses:
        models[eps] = []
        for expm in range(ne):
            X = X_s0[expm]
            T = T_s0[expm][:, None]
            model = Log_Reg(dim, 1).double()
            opt = torch.optim.LBFGS(model.parameters(), max_iter=100)
            if eps > 0: 
                b = torch.sqrt((4 * (L * R2) ** 2 * (torch.log(1 / delta) + eps / 2)) / ((eps / 2) ** 2)) * z_dist.sample((dim, 1))
                # b = torch.sqrt((8 * (torch.log(2. / delta) + 4 * eps)) / (eps ** 2)) * z_dist.sample((dim, 1))
            else:
                b = torch.zeros((dim, 1)).double()
            
            # define first-order oracle for lbfgs
            def closure():
                if torch.is_grad_enabled():
                    opt.zero_grad()
                outputs = model(X)
                if eps > 0:
                    for weights in model.parameters():
                        reg_noise = 1 / nf * torch.matmul(weights, b) + 0.5 * reg_co * weights.norm(2).pow(2)
                        # reg_noise = 1 / nf * torch.matmul(weights, b) + 0.5 * (2 * c * X_std / (eps * nf) + reg_co) * weights.norm(2).pow(2)
                else:
                    for weights in model.parameters():
                        reg_noise = 0.5 * reg_co * weights.norm(2).pow(2)
                loss = torch.nn.functional.binary_cross_entropy_with_logits(outputs, T) + reg_noise
                if loss.requires_grad:
                    loss.backward()
                return loss
            
            opt.step(closure)

            models[eps].append(model)
      
    #############################
    # estimate treatment effect #
    #############################

    # initialise pi_hat dictionaries
    pi_hats = {}
    
    # initialise e dictionary
    e = {}
    
    # intialise sigma dictionary
    sig_d = {}

    # get estimated propensity scores
    pi_hats[0] = torch.stack(
        [models[0][i](X_s1[i]).squeeze() for i in range(ne)]
    )

    for eps in epses[1:]:
        # get perturbed propensity scores
        pi_hats[eps] = torch.stack(
            [models[eps][i](X_s1[i]).squeeze() for i in range(ne)]
        )
                
        # max of abs of Y1_s1 / propensity score for each experiment
        max_abs_Y1_s1_div_ps = torch.max(
            torch.abs(Y1_s1) / ((ns - nf) * pi_hats[eps]), 1
        )[0]
                
        # max of abs of Y0_s1 / (1 - propensity score) for each experiment
        max_abs_Y0_s1_div_1_m_ps = torch.max(
            torch.abs(Y0_s1) / ((ns - nf) * (1 - pi_hats[eps])), 1
        )[0]
        
        # hstack max_abs_Y_s1_div_ps and max_abs_Y_s1_div_1_m_ps
        max_abs_all = torch.stack(
            (max_abs_Y1_s1_div_ps, max_abs_Y0_s1_div_1_m_ps), 1
        )
                
        # replace inf/nan with 1e20 for stability
        max_abs_all[torch.isfinite(max_abs_all) == 0] = 1e20
            
        # define sensitivity for estimation
        s_e = 2 * torch.max(max_abs_all, 1)[0]
        
        # define sigma for estimation
        sigma_e = np.sqrt(
            2 * np.log(1.25 / delta) + 1e-10
        ) * (s_e / (eps / 2))
        sig_d[eps] = sigma_e.detach().numpy()
        sigma_e_2 = sigma_e ** 2
        
#         # analytic gaussian mechanism
#         sigma_e = calibrateAnalyticGaussianMechanism(eps, delta, s_e)
#         sigma_e_2 = sigma_e ** 2

        # define e distribution for estimation
        e_dist = torch.distributions.multivariate_normal.MultivariateNormal(
            torch.tensor([0.0], dtype=torch.float64),
            torch.diag(sigma_e)
        )

        # draw e for estimation
        e[eps] = e_dist.sample().reshape(ne)
    
    # get treatment effects
    # true
    te = {}
    # empirical means and std of means of ERM + private ERM
    te_hats = {'means': [], 'stds': []}
    # means and std of means of privatised te_hats
    te_hats_p = {'means': [], 'stds': []}
             
    for key in pi_hats.keys():
        # reduce_mean from (ne, est_split) tensor to (ne, 1) matrix
        te_hats_ = torch.mean(
            Y1_s1 / pi_hats[key] - Y0_s1 / (1 - pi_hats[key]), 
            1,
        )
        te_hats['means'].append(
            te_hats_.detach().numpy()
        )
        te_hats['stds'].append(
            te_hats_.std().detach().numpy()
        )
        try:
            te_hats_p_ = te_hats_ + e[key]
            te_hats_p['means'].append(
                te_hats_p_.detach().numpy()
            )
            te_hats_p['stds'].append(
                te_hats_p_.std().detach().numpy()
            )
        except KeyError:
            # fill first row for later
            te_hats_p['means'].append(
                te_hats_.detach().numpy()
            )
            te_hats_p['stds'].append(
                te_hats_.std().detach().numpy()
            )
        
    te_hats['means'] = np.array(te_hats['means'])
    te_hats['stds'] = np.array(te_hats['stds'])
    te_hats_p['means'] = np.array(te_hats_p['means'])
    te_hats_p['stds'] = np.array(te_hats_p['stds'])

    return te, te_hats, te_hats_p, sig_d

## Run method and print results

In [9]:
te, te_hats, te_hats_p, sig_d = IPW_PPS_Out(X_all, T_all, Y_all, epses, delta, reg_co, nf)

/Users/sikai/.pyenv/versions/3.7.6/lib/python3.7/site-packages/ipykernel_launcher.py:109: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


In [10]:
means = [np.mean(i) for i in te_hats['means']]
se = [np.std(i) / np.sqrt(ne) for i in te_hats['means']]

for i in range(len(means)):
    if i == 0:
        print('The mean ATE for no epsilon is {} and its s.e. is {}'.format(means[i], se[i]))
    else:
        print('The non-privatised mean ATE for epsilon = {} is {} and its s.e. is {}'.format(epses[i-1], means[i], se[i]))                        

The mean ATE for no epsilon is 15.033430336156096 and its s.e. is 0.35819745636166966
The non-privatised mean ATE for epsilon = 0 is -inf and its s.e. is nan
The non-privatised mean ATE for epsilon = 0.2 is 563267.6525562855 and its s.e. is 613395.4503160334
The non-privatised mean ATE for epsilon = 0.4 is 817.7294752865399 and its s.e. is 314.4303943779342
The non-privatised mean ATE for epsilon = 0.6 is 67.20964227857449 and its s.e. is 24.674317498045625
The non-privatised mean ATE for epsilon = 0.8 is 54.31971501471241 and its s.e. is 7.455008276797289


/Users/sikai/.pyenv/versions/3.7.6/lib/python3.7/site-packages/numpy/core/_methods.py:202: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)


In [11]:
means = [np.mean(i) for i in te_hats_p['means']]
se = [np.std(i) / np.sqrt(ne) for i in te_hats_p['means']]

for i in range(len(means)):
    if i == 0:
        print('The mean ATE for no epsilon is {} and its s.e. is {}'.format(means[i], se[i]))
    else:
        print('The privatised mean ATE for epsilon = {} is {} and its s.e. is {}'.format(epses[i-1], means[i], se[i]))                        

The mean ATE for no epsilon is 15.033430336156096 and its s.e. is 0.35819745636166966
The privatised mean ATE for epsilon = 0 is -inf and its s.e. is nan
The privatised mean ATE for epsilon = 0.2 is 563243.0776508197 and its s.e. is 613376.969479873
The privatised mean ATE for epsilon = 0.4 is 817.8445000575667 and its s.e. is 315.32626946345334
The privatised mean ATE for epsilon = 0.6 is 66.50325492294587 and its s.e. is 24.698620124944828
The privatised mean ATE for epsilon = 0.8 is 54.32102945682354 and its s.e. is 7.501413817779162


In [12]:
sgn_tau_hat = np.sign(te_hats['means'][0])
    
# compute probabilities
probs = [sum(np.sign(i) != sgn_tau_hat) / ne for i in te_hats['means'][1:]]

for i in range(1, len(epses)):
    print('The probability of signs being flipped for non-privatised ATE for epsilon = {} is {}'.format(epses[i], probs[i-1]))     

The probability of signs being flipped for non-privatised ATE for epsilon = 0.2 is 0.528
The probability of signs being flipped for non-privatised ATE for epsilon = 0.4 is 0.41
The probability of signs being flipped for non-privatised ATE for epsilon = 0.6 is 0.314
The probability of signs being flipped for non-privatised ATE for epsilon = 0.8 is 0.231
The probability of signs being flipped for non-privatised ATE for epsilon = 0.99 is 0.176


In [13]:
sgn_tau_hat = np.sign(te_hats_p['means'][0])
    
# compute probabilities
probs = [sum(np.sign(i) != sgn_tau_hat) / ne for i in te_hats_p['means'][1:]]

for i in range(1, len(epses)):
    print('The probability of signs being flipped for privatised ATE for epsilon = {} is {}'.format(epses[i], probs[i-1]))     

The probability of signs being flipped for privatised ATE for epsilon = 0.2 is 0.529
The probability of signs being flipped for privatised ATE for epsilon = 0.4 is 0.413
The probability of signs being flipped for privatised ATE for epsilon = 0.6 is 0.303
The probability of signs being flipped for privatised ATE for epsilon = 0.8 is 0.23
The probability of signs being flipped for privatised ATE for epsilon = 0.99 is 0.181


In [14]:
te, te_hats, te_hats_p, sig_d = IPW_PPS_Obj(X_all, T_all, Y_all, epses, delta, reg_co, nf)

In [15]:
means = [np.mean(i) for i in te_hats['means']]
se = [np.std(i) / np.sqrt(ne) for i in te_hats['means']]

for i in range(len(means)):
    if i == 0:
        print('The mean ATE for no epsilon is {} and its s.e. is {}'.format(means[i], se[i]))
    else:
        print('The non-privatised mean ATE for epsilon = {} is {} and its s.e. is {}'.format(epses[i-1], means[i], se[i]))                        

The mean ATE for no epsilon is 15.03343853707586 and its s.e. is 0.35819117126040745
The non-privatised mean ATE for epsilon = 0 is -3873168066.3329234 and its s.e. is 3820312742.8553967
The non-privatised mean ATE for epsilon = 0.2 is 444.3367602418289 and its s.e. is 199.1621070492235
The non-privatised mean ATE for epsilon = 0.4 is 67.13478969965367 and its s.e. is 10.01034770468249
The non-privatised mean ATE for epsilon = 0.6 is 28.00095553344574 and its s.e. is 1.4673441398598515
The non-privatised mean ATE for epsilon = 0.8 is 22.870310416137585 and its s.e. is 0.9651547074460028


In [16]:
means = [np.mean(i) for i in te_hats_p['means']]
se = [np.std(i) / np.sqrt(ne) for i in te_hats_p['means']]

for i in range(len(means)):
    if i == 0:
        print('The mean ATE for no epsilon is {} and its s.e. is {}'.format(means[i], se[i]))
    else:
        print('The privatised mean ATE for epsilon = {} is {} and its s.e. is {}'.format(epses[i], means[i], se[i]))                        

The mean ATE for no epsilon is 15.03343853707586 and its s.e. is 0.35819117126040745
The privatised mean ATE for epsilon = 0.2 is -3873162677.143155 and its s.e. is 3820306794.0853395
The privatised mean ATE for epsilon = 0.4 is 445.35454784502053 and its s.e. is 199.18413333398016
The privatised mean ATE for epsilon = 0.6 is 66.9816377206565 and its s.e. is 9.890741783750363
The privatised mean ATE for epsilon = 0.8 is 27.875289015922935 and its s.e. is 1.4742573852736496
The privatised mean ATE for epsilon = 0.99 is 22.78265811464902 and its s.e. is 0.9592348910508157


In [17]:
sgn_tau_hat = np.sign(te_hats['means'][0])
    
# compute probabilities
probs = [sum(np.sign(i) != sgn_tau_hat) / ne for i in te_hats['means'][1:]]

for i in range(1, len(epses)):
    print('The probability of signs being flipped for non-privatised ATE for epsilon = {} is {}'.format(epses[i], probs[i-1]))     

The probability of signs being flipped for non-privatised ATE for epsilon = 0.2 is 0.528
The probability of signs being flipped for non-privatised ATE for epsilon = 0.4 is 0.34
The probability of signs being flipped for non-privatised ATE for epsilon = 0.6 is 0.143
The probability of signs being flipped for non-privatised ATE for epsilon = 0.8 is 0.056
The probability of signs being flipped for non-privatised ATE for epsilon = 0.99 is 0.015


In [19]:
sgn_tau_hat = np.sign(te_hats_p['means'][0])
    
# compute probabilities
probs = [sum((sgn_tau_hat != np.sign(te_hats['means'][1:][i])).astype('int') + 
            (sgn_tau_hat != np.sign(te_hats_p['means'][1:][i])).astype('int') == 2) / ne
         for i in range(len(te_hats['means'][1:]))]

for i in range(1, len(epses)):
    print('The probability of signs being flipped for privatised ATE for epsilon = {} is {}'.format(epses[i], probs[i-1]))     

The probability of signs being flipped for privatised ATE for epsilon = 0.2 is 0.52
The probability of signs being flipped for privatised ATE for epsilon = 0.4 is 0.321
The probability of signs being flipped for privatised ATE for epsilon = 0.6 is 0.13
The probability of signs being flipped for privatised ATE for epsilon = 0.8 is 0.046
The probability of signs being flipped for privatised ATE for epsilon = 0.99 is 0.011
